NOTE: This one does not work, b/c it only pulls recent performance *with respect to the pull date* and no retroactive perf data. So this one is dead

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import pandas as pd
import requests
import urllib.request
import numpy as np
from datetime import date
import time

from bs4 import BeautifulSoup

In [3]:
# Get team codes
team_codes = pd.read_csv('team_codes.csv')
gl_codes = team_codes['game_log_code'].tolist()
codes = team_codes['code'].tolist()


## Make last 30 days df
df = pd.DataFrame()
days = 30

# NOTE: this is scaled back to just a few teams for testing
for i in range(round(0.05*len(codes))):

    # Loop through teams
    team = codes[i]
    gl_team = gl_codes[i]

    # Request setup
    url = f'https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=b&dates=lastndays&lastndays={days}&since=2023-09-01&fromandto=2023-09-01.2023-09-30&level=mlb&franch={gl_team}'

    team_df = pd.read_html(url)
    team_df = pd.DataFrame((team_df[0]))

    # Add team identifier
    team_df['Team'] = team

    # Append into overall team dataframe
    df = df.append(team_df)
    
    # Print status
    print(f'Done: {team}')

    # Wait for 3.2 seconds, b/c baseball-reference has a 20 request/min limit
    time.sleep(3.2)

## Now that we have scraped all the raw data, we can prep it

# Narrow into team counting perf stats
df = df[['Team', 'Tm', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP']]

# Drop Nan
df = df.dropna()

# Drop rows where 'Tm' has multiple teams due to trade (rare situation)
df = df[df['Tm'].str.contains(',')==False]

# Drop rows where 'Tm' has literally 'Tm' due to rows with column headers in middle of table
df = df[df['Tm'].str.contains('Tm')==False]

# Cast datatypes to numeric
df = df.apply(pd.to_numeric, errors='ignore')

# Calculate total bases column
# Note, this is a progressive calculation including walks etc as a base
df['TB'] = df['H']+df['3B']+2*df['HR']+df['BB']+df['IBB']+df['HBP']

# Get team totals
df_grouped = df.groupby('Team',as_index=False).sum()

# Calculate team rate stats
df_grouped['OBP'] = (df_grouped['H']+df_grouped['BB']+df_grouped['IBB']+df_grouped['HBP'])/df_grouped['PA']
df_grouped['AVG'] = df_grouped['H']/df_grouped['AB']
df_grouped['SLG'] = df_grouped['TB']/df_grouped['AB']
df_grouped['OPS'] = df_grouped['OBP']+df_grouped['SLG']

team_L30 = df_grouped[['Team', 'PA', 'OBP', 'AVG', 'SLG', 'OPS']]

# Rename cols to identify L30
team_L30.columns = ['Team', 'PA_L30', 'OBP_L30', 'AVG_L30', 'SLG_L30', 'OPS_L30']


## Make season opponent offensive performance df
# Request setup
url = f'https://www.baseball-reference.com/leagues/majors/2023.shtml'

team_all = pd.read_html(url)
team_all = pd.DataFrame(team_all[0])

# Add in useable team name
team_all = team_all.merge(team_codes, left_on = 'Tm', right_on = 'team')

# Select and rename columns
team_all = team_all[['code','PA', 'BA', 'OBP', 'SLG', 'OPS']]
team_all.columns = ['Team','PA_all', 'BA_all', 'OBP_all', 'SLG_all', 'OPS_all']


## Combine L30 and full season dfs
df = team_L30.merge(team_all)

# Export to csv
df.to_csv('opp_perf.csv', index = False)

df



Done: ARI
Done: ATL


,Team,PA_L30,OBP_L30,AVG_L30,SLG_L30,OPS_L30,PA_all,BA_all,OBP_all,SLG_all,OPS_all
0,ARI,1046,0.326004,0.249458,0.446855,0.772858,5400,.252,.323,.417,.740
1,ATL,1090,0.356881,0.288481,0.515800,0.872681,5411,.276,.345,.502,.847


In [ ]:
'https://www.baseball-reference.com/leagues/daily.fcgi?\
request=1&type=b&lastndays=1&dates=since&since=2023-09-01\
&fromandto=2023-09-01.2023-09-30&level=mlb&franch=ANY'


